In [ ]:
from IPython.core.display import HTML
HTML("<style>.container { width:95% !important; }</style>")

# Lecture 6: Indirect methods for constrained optimization

## Some remarks
* How to deal with problems where the objective function should be maximized, i.e. $\max f(x)$?
  * We can use the same methods if we instead minimize the negative of $f$, i.e. $\min -f(x)$
  * The optimal solution $x^*$ is the same for both the problems

## Simple example

In [ ]:
def f_max(x):
    return -(x-3.0)**2 + 10.0

# clearly x* = 3.0 is the global maximum  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.arange(-5.0, 12.0, 0.3)
plt.plot(x, f_max(x), 'bo')
plt.show()
#print(f1(0.0))

In [ ]:
from scipy.optimize import minimize_scalar

# multiply f_max with -1.0
def g(x):
    return -f_max(x)

plt.plot(x, g(x), 'ro')
plt.show()

In [ ]:
res = minimize_scalar(g,method='brent')
print(res)
print(g(res.x))
print(f_max(res.x))

# Constrained optimization

Now we will move to studying constrained optimization problems i.e., the full problem
$$
\begin{align} \
\min \quad &f(x)\\
\text{s.t.} \quad & g_j(x) \geq 0\text{ for all }j=1,\ldots,J\\
& h_k(x) = 0\text{ for all }k=1,\ldots,K\\
&a_i\leq x_i\leq b_i\text{ for all } i=1,\ldots,n\\
&x\in \mathbb R^n,
\end{align}
$$
where for all $i=1,\ldots,n$ it holds that $a_i,b_i\in \mathbb R$ or they may also be $-\infty$ or $\infty$.

## On optimal solutions for constrained problems
* Two types of constraints: equality and inequality constraints
* Inequality constraint $g_i(x)\geq0$ is said to be *active* at point $x$ if $g_i(x)=0$
* Linear constraints are much easier to consider --> their gradients are constant
* Nonlinear constraints trickier --> gradient changes for different values of decision variables

No constraints
![](images/unconstrained.png)
<span style="font-size: 10pt;">*Adopted from Prof. L.T. Biegler (Carnegie Mellon University)*</span>

Inequality constraints
![](images/inequality_constraints.jpg)
<span style="font-size: 10pt;">*Adopted from Prof. L.T. Biegler (Carnegie Mellon University)*</span>

Both inequality and equality constraints
![](images/constraints.jpg)
<span style="font-size: 10pt;">*Adopted from Prof. L.T. Biegler (Carnegie Mellon University)*</span>

## Transforming the constraints
Type of inequality:
$$
g_i(x)\geq0 \iff -g_i(x)\leq0
$$


Inequality to equality:
$$
g_i(x)\leq0 \iff g_i(x)+y_i^2=0
$$
* $y_i$ is a *slack variable*; constraint is active if $y_i=0$
* By adding $y_i^2$ no need to add $y_i\geq0$
* If $g$ is linear, linearity can be preserved by $g_i(x)+y_i=0, y_i\geq0$

Equality to inequality:
$$
h_i(x)=0 \iff h_i(x)\geq0 \text{ and } -h_i(x) \geq0
$$

## Example problem
For example, we can have an optimization problem
$$
\begin{align} \
\min \quad &x_1^2+x_2^2\\
\text{s.t.} \quad & x_1+x_2-1\geq 0\\
&-1\leq x_1\leq 1, x_2\leq 3.\\
\end{align}
$$

In order to optimize that problem, we can define the following python function:

In [ ]:
import numpy as np
def f_constrained(x):
    return np.linalg.norm(x)**2,[x[0]+x[1]-1],[]

Now, we can call the function:

In [ ]:
(f_val,ieq,eq) = f_constrained([1,0])
print("Value of f is "+str(f_val))
if len(ieq)>0:
    print("The values of inequality constraints are:")
    for ieq_j in ieq:
        print(str(ieq_j)+", ")
if len(eq)>0:
    print("The values of the equality constraints are:")
    for eq_k in eq:
        print(str(eq_k)+", ")

Is this solution feasible?

In [ ]:
if all([ieq_j>=0 for ieq_j in ieq]) and all([eq_k==0 for eq_k in eq]):
    print("Solution is feasible")
else:
    print("Solution is infeasible")

# Indirect and direct methods for constrained optimization

There are two categories of methods for constrained optimization: Indirect and direct methods. The main difference is that
1. Indirect methods convert the constrained optimization problem into a single or a sequence of unconstrained optimization problems, that are then solved. Often, the intermediate solutions do not need to be feasible, but the sequence of solutions converges to a solution that is optimal for the original problem (and, thus, feasible).

2. Direct methods deal with the constrained optimization problem directly. In this case, all the intermediate solutions are feasible.

# Indirect methods

## Penalty function methods

**IDEA:** Include constraints into the objective function with the help of penalty functions that penalize constraint violations.

Let, $\alpha(x):\mathbb R^n\to\mathbb R$ be a function so that 
* $\alpha(x)= 0$, for all feasible $x$
* $\alpha(x)>0$, for all infeasible $x$.

Define a set of optimization problems (depending on parameter $r$)
$$
\begin{align} \
\min \qquad &f(x)+r\alpha(x)\\
\text{s.t.} \qquad &x\in \mathbb R^n
\end{align}
$$
for $r>0$. Let $x_r$ be an optimal solution of such problem for a given $r$.

In this case, the optimal solutions $x_r$ converge to the optimal solution of the constrained problem, when $r\to\infty$, if such a solution exists.

For example, good ideas for penalty functions are
* $h_k(x)^2$ for equality constraints,
* $\left(\min\{0,g_j(x)\}\right)^2$ for inequality constraints.

# Illustrative example
$$
\min x \\
\text{ s.t. } -x + 2 \leq 0
$$
Let
$$
\alpha(x) = \{(\max[0,(-x+2)])^2, 0 \text{ if }x\geq2\}
$$
Then 
$$
\alpha(x) = (-x+2)^2, \text{ if } x<2
$$
Minimum of $f+r\alpha(x)$ is at $2-1/2r$
![](images/penalty.png)

In [ ]:
def alpha(x,f):
    (_,ieq,eq) = f(x)
    return sum([min([0,ieq_j])**2 for ieq_j in ieq])+sum([eq_k**2 for eq_k in eq])

In [ ]:
alpha([1,0],f_constrained)

In [ ]:
def penalized_function(x,f,r):
    return f(x)[0] + r*alpha(x,f)

In [ ]:
print(penalized_function([-1,0],f_constrained,10000))
print(penalized_function([-1,0],f_constrained,100))
print(penalized_function([-1,0],f_constrained,10))

Let's solve the penalty problem by using Nelder-Mead from scipy.optimize

In [ ]:
from scipy.optimize import minimize
res = minimize(lambda x:penalized_function(x,f_constrained,1),
               [0,0],method='Nelder-Mead', 
         options={'disp': True})
print(res.x)

In [ ]:
(f_val,ieq,eq) = f_constrained(res.x)
print("Value of f is "+str(f_val))
if len(ieq)>0:
    print("The values of inequality constraints are:")
    for ieq_j in ieq:
        print(str(ieq_j)+", ")
if len(eq)>0:
    print("The values of the equality constraints are:")
    for eq_k in eq:
        print(str(eq_k)+", ")

if all([ieq_j>=0 for ieq_j in ieq]) and all([eq_k==0 for eq_k in eq]):
    print("Solution is feasible")
else:
    print("Solution is infeasible")

### How to set the penalty term $r$?

The penalty term should
* be large enough in order for the solutions be close enough to the feasible region, but
* not be too large to
  * cause numerical problems, or
  * cause premature convergence to non-optimal solutions because of relative tolerances.

Usually, the penalty term is either
* set as big as possible without causing problems (hard to know), or
* updated iteratively.


**Note:** 

* We solved our example problem with a fixed value for the penalty parameter $r$. In order to make the penalty function method work in practice, you have to implement the iterative update for $r$. This you can practice in one of the upcoming exercises!

* The starting point for solving the penalty problems can be selected in an efficient way. When you set $r_i$ and solve the corresponding unconstrained penalty problem, you get an optimal solution $x_{r_i}$. Then you update $r_i\rightarrow r_{i+1}$ and you can use $x_{r_i}$ as a starting point for solving the penalty problem with $r_{i+1}$.

# Barrier function methods

**IDEA:** Prevent leaving the feasible region so that the value of the objective is $\infty$ outside the feasible set.

This method is only applicable to problems with inequality constraints and for which the set 
$$\{x\in \mathbb R^n: g_j(x)>0\text{ for all }j=1,\ldots,J\}$$
is non-empty.

Let $\beta:\{x\in \mathbb R^n: g_j(x)>0\text{ for all }j=1,\ldots,J\}\to \mathbb R$ be a function so that $\beta(x)\to \infty$, when $x\to\partial\{x\in \mathbb R^n: g_j(x)>0\text{ for all }j=1,\ldots,J\}$, where $\partial A$ is the boundary of the set $A$. 

Now, define optimization problem 
$$
\begin{align}
\min \qquad & f(x) + r\beta(x)\\
\text{s.t. } \qquad & x\in \{x\in \mathbb R^n: g_j(x)>0\text{ for all }j=1,\ldots,J\}.
\end{align}
$$
and let $x_r$ be the optimal solution of this problem (which we assume to exist for all $r>0$).

In this case, $x_r$ converges to the optimal solution of the problem (if it exists), when $r\to 0^+$ (i.e., $r$ converges to zero from the right side (= positive numbers)).

A good idea for a barrier function is $\frac1{g_j(x)}$.

In [ ]:
def beta(x,f):
    _,ieq,_ = f(x)
    try:
        value=sum([1/max([0,ieq_j]) for ieq_j in ieq])
    except ZeroDivisionError:
        value = float("inf")
    return value 

In [ ]:
def function_with_barrier(x,f,r):
    return f(x)[0]+r*beta(x,f)

In [ ]:
# let's try to find a feasible starting point
print(f_constrained([1,1]))

In [ ]:
from scipy.optimize import minimize
res = minimize(lambda x:function_with_barrier(x,f_constrained,0.0000001),
               [1,1],method='Nelder-Mead', options={'disp': True})
print(res.x)

In [ ]:
(f_val,ieq,eq) = f_constrained(res.x)
print("Value of f is "+str(f_val))
if len(ieq)>0:
    print("The values of inequality constraints are:")
    for ieq_j in ieq:
        print(str(ieq_j)+", ")
if len(eq)>0:
    print("The values of the equality constraints are:")
    for eq_k in eq:
        print(str(eq_k)+", ")
if all([ieq_j>=0 for ieq_j in ieq]) and all([eq_k==0 for eq_k in eq]):
    print("Solution is feasible")
else:
    print("Solution is infeasible")

It is 'easy' to see that x* = (0.5,0.5) 

https://www.wolframalpha.com/input/?i=minimize+x%5E2%2By%5E2+on+x%2By%3E%3D1

In [ ]:
print(f_constrained([.5,.5]))

## Other notes about using penalty and barrier function methods

* It is worthwile to consider whether feasibility can be compromized. If the constraints do not have any tolerances, then barrier function method should be considered.

* Also barrier methods parameter can be set iteratively

* Penalty and barrier functions should be chosen so that they are differentiable (thus $x^2$ above)

* In both methods, the minimum is attained at the limit.

* Different penalty and barrier parameters can be used for different constraints, even for same problem.